|        |         |                                @ |
|:-------|:--------|---------------------------------:|
| Luca   | Mosetti | luca.mosetti-1@studenti.unitn.it |
| Shandy | Darma   |   shandy.darma@studenti.unitn.it |

In [ ]:
from dataclasses import dataclass
from typing import Iterator, Callable, Iterable, SupportsFloat
from matplotlib_inline.backend_inline import set_matplotlib_formats
from numpy.typing import NDArray

import doctest
import math
import matplotlib
import heapq as hq
import itertools as it
import statistics as st
import matplotlib.pyplot as plt
import more_itertools as mit
import numpy as np
import scipy as sp

In [ ]:
set_matplotlib_formats('svg')

# Exercise 2

1. Repeat exercise 1 for an $\text M/\text M/c/K$ system where

    - there are $c$ servers (start with $c = 2$, then test for larger values if time allows);
    - the queue of each server can hold up to $K$ packets (start with some easy number such as $K = 10$): this means that a server having $K$ packets in queue would discard any other packets assigned to it.

2. Experiment with simple packet assignment policies (round-robin, least-loaded servers first, ...) as well as with policies that consider the occupancy of the queue of the servers (e.g., avoid sending packets to servers that have a full queue, or to a queue that is more than $x\%$ full, for some $x$).

3. For each policy, plot a histogram showing the number of packets served by each of the $c$ servers, as well as the average distribution of the queuing delay experienced by each packet. You can show other metrics as needed.

In [ ]:
@dataclass(order=True, frozen=True, slots=True)
class START:
    pass


@dataclass(order=True, frozen=True, slots=True)
class ARRIVAL:
    pass


@dataclass(order=True, frozen=True, slots=True)
class DEPARTURE:
    by: int


@dataclass(order=True, frozen=True, slots=True)
class ASSIGNED:
    by: int


@dataclass(order=True, frozen=True, slots=True)
class SERVING:
    by: int


@dataclass(order=True, frozen=True, slots=True)
class DISCARDED:
    by: int


ET = START | ARRIVAL | DEPARTURE
LT = ASSIGNED | SERVING | DEPARTURE | DISCARDED


@dataclass(order=True, frozen=True, slots=True)
class Event:
    timestamp: float
    event: ET


@dataclass(order=True, frozen=True, slots=True)
class Log:
    timestamp: float
    log: LT

$$
U \sim \text{Uniform}(0, 1) \qquad X = - \frac {\log U} \lambda \sim \text{Exp}(\lambda)
$$

In [ ]:
def uni_to_exp(lmbd: float, u: float) -> float:
    return -math.log(u) / lmbd


def round_robin(last: int, servers: NDArray[np.uint]) -> int:
    return (last + 1) % len(servers)


def least_loaded(_: int, servers: NDArray[np.uint]) -> int:
    return np.argmin(servers).item()


def least_loaded_round_robin(last: int, servers: NDArray[np.uint]) -> int:
    """

    >>> least_loaded_round_robin(1, [0, 0, 1, 0])
    3

    >>> least_loaded_round_robin(0, [0, 1, 1, 1])
    0

    >>> zs = np.zeros(10)
    >>> all([ least_loaded_round_robin(last, zs) == round_robin(last, zs) for last in range(10) ])
    True

    >>> zs = np.zeros(10)
    >>> all([ least_loaded_round_robin(last, zs) != least_loaded(last, zs) for last in range(9) ])
    True
    """
    shift: int = last + 1
    return (np.argmin(np.roll(servers, -shift)).item() + shift) % len(servers)


def mmck_simulation(
        seed_arr: int,
        seeds_dep: list[int],
        lmbd: float,
        mu: float,
        k: int,
        policy: Callable[[int, NDArray[np.uint]], int]
) -> Iterator[Log]:
    """
    Reproducible simulation of M/M/c/K queue-server system

    >>> mit.take(100, mmck_simulation(3, [7], 1, 1, 5, least_loaded)) == mit.take(100, mmck_simulation(3, [7], 1, 1, 5, least_loaded))
    True

    >>> l1m1c1k5 = mmck_simulation(3, [7], 1, 1, 5, least_loaded)
    >>> all(a.timestamp <= b.timestamp for a, b in mit.take(100, it.pairwise(l1m1c1k5)))
    True

    >>> l1m1c1k5 = mit.take(100, mmck_simulation(3, [7], 1, 1, 5, least_loaded))
    >>> mit.quantify(l.log == DISCARDED(0) for l in l1m1c1k5) <= mit.quantify(l.log == ASSIGNED(0) for l in l1m1c1k5)
    True

    >>> l1m1c1k5 = mit.take(100, mmck_simulation(3, [7], 1, 1, 5, least_loaded))
    >>> mit.quantify(l.log == SERVING(0) for l in l1m1c1k5) <= mit.quantify(l.log == ASSIGNED(0) for l in l1m1c1k5)
    True

    >>> l1m1c1k5 = mit.take(100, mmck_simulation(3, [7], 1, 1, 5, least_loaded))
    >>> mit.quantify(l.log == DEPARTURE(0) for l in l1m1c1k5) <= mit.quantify(l.log == SERVING(0) for l in l1m1c1k5)
    True

    >>> l1m1c1k5 = mit.take(100, mmck_simulation(3, [7], 1, 1, 5, least_loaded))
    >>> l1m2c1k5 = mit.take(100, mmck_simulation(3, [7], 1, 2, 5, least_loaded))
    >>> l1m1_dep_over_arr = mit.quantify(l.log == DEPARTURE(0) for l in l1m1c1k5) / mit.quantify(l.log == ASSIGNED(0) for l in l1m1c1k5)
    >>> l1m2_dep_over_arr = mit.quantify(l.log == DEPARTURE(0) for l in l1m2c1k5) / mit.quantify(l.log == ASSIGNED(0) for l in l1m2c1k5)
    >>> l1m1_dep_over_arr < l1m2_dep_over_arr
    True

   >>> l1m1c1k3 = mit.take(100, mmck_simulation(3, [7], 1, 1, 3, least_loaded))
   >>> l1m1c1k5 = mit.take(100, mmck_simulation(3, [7], 1, 1, 5, least_loaded))
   >>> mit.quantify(l.log == DISCARDED(0) for l in l1m1c1k3) > mit.quantify(l.log == DISCARDED(0) for l in l1m1c1k5)
   True

   >>> lmc1 = mit.take(100, mmck_simulation(3, [7], 1e6, 1e-6, -1, least_loaded))
   >>> mit.quantify(l.log == DISCARDED(0) for l in lmc1)
   0
   """

    c: int = len(seeds_dep)

    assert c > 0

    rng_arr: np.random.Generator = np.random.default_rng(seed_arr)

    def next_arr(timestamp: float) -> Event:
        return Event(
            timestamp + uni_to_exp(lmbd, rng_arr.random()),
            ARRIVAL()
        )

    rngs_dep: list[np.random.Generator] = [np.random.default_rng(seed_dep) for seed_dep in seeds_dep]

    def next_dep(by: int, timestamp: float) -> Event:
        return Event(
            timestamp + uni_to_exp(mu, rngs_dep[by].random()),
            DEPARTURE(by=by)
        )

    last: int = -1
    servers: NDArray[np.uint] = np.zeros(c, np.uint)

    timeline: list[Event] = [
        Event(timestamp=0, event=START()),
    ]

    while True:
        e: Event = hq.heappop(timeline)
        match e.event:

            case START():
                hq.heappush(timeline, next_arr(e.timestamp))

            case ARRIVAL():
                hq.heappush(timeline, next_arr(e.timestamp))

                by: int = policy(last, servers.copy())
                last = by
                yield Log(e.timestamp, ASSIGNED(by=by))
                match servers[by]:
                    case 0:
                        yield Log(e.timestamp, SERVING(by=by))
                        servers[by] += 1
                        hq.heappush(timeline, next_dep(by, e.timestamp))

                    case x if x == k + 1:
                        yield Log(e.timestamp, DISCARDED(by=by))

                    case _:
                        servers[by] += 1

            case DEPARTURE(by):
                yield Log(e.timestamp, DEPARTURE(by))

                servers[by] -= 1
                if servers[by] > 0:
                    yield Log(e.timestamp, SERVING(by=by))
                    hq.heappush(timeline, next_dep(by, e.timestamp))


def waiting(c: int, xs: Iterator[Log], discarded: float | None = None) -> Iterable[tuple[float, int, float]]:
    """
    From sequence of logs to sequence of (arrival time, served by, waiting time)
    For discarded packets (arrival time, served by, discarded)

    >>> logs_0 = [Log(10, ASSIGNED(0)), Log(20, ASSIGNED(0)), Log(30, ASSIGNED(0)), Log(40, SERVING(0))]
    >>> logs_1 = [Log(15, ASSIGNED(1)), Log(18, SERVING(1)), Log(19, ASSIGNED(1)), Log(41, SERVING(1))]
    >>> logs_2 = [Log(11, ASSIGNED(2)), Log(30, SERVING(2)), Log(31, ASSIGNED(2)), Log(42, DISCARDED(2))]
    >>> logs = sorted(it.chain(logs_0, logs_1, logs_2), key=lambda l: l.timestamp)
    >>> list(waiting(3, logs))
    [(10, 0, 30), (11, 2, 19), (15, 1, 3), (19, 1, 22)]

    >>> logs_0 = [Log(10, ASSIGNED(0)), Log(20, ASSIGNED(0)), Log(30, ASSIGNED(0)), Log(40, SERVING(0))]
    >>> logs_1 = [Log(15, ASSIGNED(1)), Log(18, SERVING(1)), Log(19, ASSIGNED(1)), Log(41, SERVING(1))]
    >>> logs_2 = [Log(11, ASSIGNED(2)), Log(30, SERVING(2)), Log(31, ASSIGNED(2)), Log(42, DISCARDED(2))]
    >>> logs = sorted(it.chain(logs_0, logs_1, logs_2), key=lambda l: l.timestamp)
    >>> list(waiting(3, logs, 0))
    [(10, 0, 30), (11, 2, 19), (15, 1, 3), (19, 1, 22), (31, 2, 0)]

    >>> seeds_dep = [5, 7, 11, 13]
    >>> simulation = mmck_simulation(3, seeds_dep, 1, 1, 3, round_robin)
    >>> ws = mit.take(len(seeds_dep), waiting(len(seeds_dep), simulation))
    >>> [ (server, awaited) for _, server, awaited in ws ]
    [(0, 0.0), (1, 0.0), (2, 0.0), (3, 0.0)]

    """

    def single_waiting(by: int, xs: Iterator[Log]) -> Iterator[tuple[float, int, float]]:
        xs1, xs2 = it.tee(xs, 2)
        las = (l.timestamp for l in xs1 if l.log == ASSIGNED(by))
        lzs = ((l.log, l.timestamp) for l in xs2 if l.log in [DISCARDED(by), SERVING(by)])
        match discarded:
            case None:
                return ((a, by, e - a) for a, (l, e) in zip(las, lzs) if l != DISCARDED(by))
            case _:
                return ((a, by, e - a if l != DISCARDED(by) else discarded) for a, (l, e) in zip(las, lzs))

    xss: tuple[Iterator[Log], ...] = it.tee(xs, c)

    ws: list[Iterator[tuple[float, int, float]]] = [single_waiting(by, xss[by]) for by in range(c)]

    return hq.merge(*ws, key=lambda t: t[0])


def timestamp_packets(c: int, xs: Iterator[Log]) -> Iterator[tuple[float, NDArray[...]]]:
    """
    From sequence of logs to sequence of (timestamp, [[packets served by 0, packets discarded by 0], ...])

    >>> logs_0 = [Log(10, ASSIGNED(0)), Log(15, ASSIGNED(0)), Log(15, DISCARDED(0)), Log(25, DEPARTURE(0))]
    >>> logs_1 = [Log(11, ASSIGNED(1)), Log(20, DEPARTURE(1))]
    >>> logs_2 = [Log(12, ASSIGNED(2)), Log(30, SERVING(2)), Log(31, ASSIGNED(2)), Log(31, DISCARDED(2))]
    >>> logs = sorted(it.chain(logs_0, logs_1, logs_2), key=lambda l: l.timestamp)
    >>> list(timestamp_packets(3, logs))
    [(0, array([[0, 0],
           [0, 0],
           [0, 0]], dtype=uint64)), (10, array([[1, 0],
           [0, 0],
           [0, 0]], dtype=uint64)), (11, array([[1, 0],
           [1, 0],
           [0, 0]], dtype=uint64)), (12, array([[1, 0],
           [1, 0],
           [1, 0]], dtype=uint64)), (15, array([[1, 1],
           [1, 0],
           [1, 0]], dtype=uint64)), (20, array([[1, 0],
           [0, 0],
           [1, 0]], dtype=uint64)), (25, array([[0, 0],
           [0, 0],
           [1, 0]], dtype=uint64)), (31, array([[0, 0],
           [0, 0],
           [1, 1]], dtype=uint64))]
    """

    def tracking(acc: tuple[float, NDArray[...]], timestamp_logs: tuple[float, list[Log]]) -> tuple[
        float, NDArray[...]]:
        pckts: NDArray[...] = acc[1].copy()
        pckts[:, 1] = 0
        timestamp, ls = timestamp_logs

        for l in ls:
            match l.log:
                case ASSIGNED(by):
                    pckts[by][0] += 1
                case DEPARTURE(by):
                    pckts[by][0] -= 1
                case DISCARDED(by):
                    pckts[by][0] -= 1
                    pckts[by][1] += 1

        return timestamp, pckts

    same_timestamp: Iterator[tuple[float, list[Log]]] = it.groupby(
        (l for l in xs if isinstance(l.log, ASSIGNED | DISCARDED | DEPARTURE)),
        key=lambda l: l.timestamp
    )

    return it.accumulate(
        same_timestamp,
        tracking,
        initial=(0, np.zeros((c, 2), dtype=np.uint))
    )


def timestamp_tot_packets(c: int, xs: Iterator[Log]) -> Iterator[tuple[float, NDArray[...]]]:
    """
    From sequence of logs to sequence of (timestamp, [[tot packets served by 0, tot packets discarded by 0], ...])

    >>> logs_0 = [Log(10, ASSIGNED(0)), Log(15, ASSIGNED(0)), Log(15, DISCARDED(0)), Log(25, DEPARTURE(0))]
    >>> logs_1 = [Log(11, ASSIGNED(1)), Log(20, DEPARTURE(1))]
    >>> logs_2 = [Log(12, ASSIGNED(2)), Log(30, SERVING(2)), Log(31, ASSIGNED(2)), Log(31, DISCARDED(2))]
    >>> logs = sorted(it.chain(logs_0, logs_1, logs_2), key=lambda l: l.timestamp)
    >>> list(timestamp_tot_packets(3, logs))
    [(0, array([[0, 0],
           [0, 0],
           [0, 0]], dtype=uint64)), (10, array([[1, 0],
           [0, 0],
           [0, 0]], dtype=uint64)), (11, array([[1, 0],
           [1, 0],
           [0, 0]], dtype=uint64)), (12, array([[1, 0],
           [1, 0],
           [1, 0]], dtype=uint64)), (15, array([[1, 1],
           [1, 0],
           [1, 0]], dtype=uint64)), (31, array([[1, 1],
           [1, 0],
           [1, 1]], dtype=uint64))]
    """

    def tracking(acc: tuple[float, NDArray[...]], timestamp_logs: tuple[float, list[Log]]) -> tuple[
        float, NDArray[...]]:
        pckts: NDArray[...] = acc[1].copy()
        timestamp, ls = timestamp_logs

        for l in ls:
            match l.log:
                case ASSIGNED(by):
                    pckts[by][0] += 1
                case DISCARDED(by):
                    pckts[by][0] -= 1
                    pckts[by][1] += 1

        return timestamp, pckts

    same_timestamp: Iterator[tuple[float, list[Log]]] = it.groupby(
        (l for l in xs if isinstance(l.log, ASSIGNED | DISCARDED)),
        key=lambda l: l.timestamp
    )

    return it.accumulate(
        same_timestamp,
        tracking,
        initial=(0, np.zeros((c, 2), dtype=np.uint))
    )


def non_overlapping_batches(xs: Iterator[SupportsFloat | tuple[SupportsFloat, ...]], b: int, m: int) -> NDArray[...]:
    """
    From sequence of tuple to non-overlapping batches NDArray

    >>> non_overlapping_batches([(1.5, 1), (2.5, 2), (3.5, 3), (4.5, 4), (5.5, 5)], 2, 2)
    array([[[1.5, 1. ],
            [2.5, 2. ]],
    <BLANKLINE>
           [[3.5, 3. ],
            [4.5, 4. ]]])
    """
    vs: NDArray[...] = np.asarray(mit.take(m * b, xs))
    return np.stack([vs[i * m:(i + 1) * m] for i in range(b)])


def gamma() -> float:
    return 0.95


def populate(
        a: plt.Axes,
        b: int,
        m: int,
        mus: NDArray[float],
        grand_mean: float,
        seeds: tuple[int | list[int], ...],
        lmbd: float,
        mu: float,
        k: int,
        delta: float,
        y_label: str,
        policy: str,
        color: int
) -> None:
    a.hlines(
        y=mus,
        xmin=np.arange(0, b) * m,
        xmax=(np.arange(0, b) + 1) * m,
        colors=[f'C{i}' for i in range(b)],
        alpha=0.2,
    )

    a.axhspan(
        grand_mean - delta,
        grand_mean + delta,
        alpha=0.5,
        color=f'C{color}',
        label=f'CI {gamma()}'
    )
    a.axhline(
        grand_mean,
        label=r'$\hat\theta$',
        color=f'C{color}',
    )

    a.legend(loc='upper left')
    a.set_title(
        f'${seeds} \\vdash \\lambda={lmbd}, \\mu={mu}, k={k}$ // non-overlapping $m = {m}, b = {b}$ // {policy}')
    a.set_xlabel('samples')
    a.set_ylabel(y_label)

In [ ]:
doctest.testmod()

In [ ]:
seeds: Iterator[int] = mit.sieve(1_000)
mit.consume(seeds, 1)

For this simulation, we are going to implement 3 different policies:

- Round Robin: This policy will fill the next queue incrementally. If it reaches the last queue, it will return to the first queue.
- Least Loaded: This policy will find the queue with the least load. Assignment is not random, it will always start searching from the first queue.
- Least Loaded Round Robin: This policy behaves like round robin, but instead it finds the next queue with least load.

In [ ]:
c: int = 2
k: int = 10
lmbd: float = 95 * c
mu: float = 100

policies: dict[str, Callable] = {
    'round robin': round_robin,
    'least loaded': least_loaded,
    'least loaded round robin': least_loaded_round_robin,
}

In [ ]:
seed_arr: int = next(seeds)
seeds_dep: list[int] = mit.take(c, seeds)

## Waiting times

First, we are going to measure the time it takes from a packet from the beginning of a queue to be processed

In [ ]:
a: plt.Axes
f, axs = plt.subplots(len(policies), 1, figsize=(12, 5 * len(policies) + 2), sharex='all', sharey='all')

for i, a, (name, policy) in zip(it.count(), axs, policies.items()):
    ws: Iterable[tuple[float, int, float]] = waiting(
        c,
        mmck_simulation(seed_arr, seeds_dep, lmbd, mu, k, policy),
        -0.025,
    )

    samples: list[tuple[float, int, float]] = mit.take(1_000, ws)

    a.bar(
        range(1, len(samples) + 1),
        [w for _, _, w in samples],
        color=[f'C{server}' for _, server, w in samples],
        hatch=['\\' * 5 if w < 0 else '' for _, server, w in samples],
    )

    a.grid(True, axis='y')
    a.set_axisbelow(True)
    a.set_xlabel('packet')
    a.set_ylabel('time')
    a.set_title(f'$({seed_arr}, {seeds_dep}) \\vdash \\lambda={lmbd}, \\mu={mu}, k={k}$ // {name}')

    if i == 0:
        a.legend(
            loc='upper left',
            handles=[
                        matplotlib.patches.Patch(
                            color=f'C{server}',
                            label=f'awaited in #{server + 1}'
                        )
                        for server in range(c)
                    ] + [
                        matplotlib.patches.Patch(
                            edgecolor='black',
                            facecolor=f'C{server}',
                            hatch='\\' * 5,
                            label=f'discarded by #{server + 1}',
                        )
                        for server in range(c)
                    ],
        )

pass

## Packets in the system over time

In this section, we will see how many packets are in the system over time.
The packets we measure are both in process and in queue.

In [ ]:
a: plt.Axes
f, axs = plt.subplots(len(policies), 1, figsize=(12, 5 * len(policies) + 2), sharex='all', sharey='all')

for i, a, (name, policy) in zip(it.count(), axs, policies.items()):
    timestamps_packets: Iterable[tuple[float, NDArray[...]]] = timestamp_packets(
        c,
        mmck_simulation(seed_arr, seeds_dep, lmbd, mu, k, policy)
    )

    samples: list[tuple[float, NDArray[...]]] = mit.take(500, timestamps_packets)
    timestamps, pckts = zip(*samples)
    timespans: NDArray[float] = np.asarray([t2 - t1 for t1, t2 in it.pairwise(it.chain([0], timestamps))], float)

    xs: NDArray[int] = np.arange(len(pckts)) + 1
    width: float = 0.5 / c

    for server in range(c):
        a.stairs(
            [snapshot[server, 0] for snapshot in pckts],
            edges=np.cumsum(np.append(0, timespans)),
            color=f'C{server}',
            alpha=0.5,
            fill=True,
        )
        a.bar(
            timestamps,
            [snapshot[server, 1] * -1 for snapshot in pckts],
            width=0.001,
            facecolor='white',
            edgecolor=f'C{server}',
            alpha=0.5,
            hatch='\\' * 5,
        )

    a.axhline(
        0,
        color='red',
        linestyle='--'
    )

    a.axhline(
        k + 1,
        color='red',
        linestyle='--'
    )

    a.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
    a.set_xlabel('time')
    a.set_ylabel('packets')
    a.set_title(f'$({seed_arr}, {seeds_dep}) \\vdash \\lambda={lmbd}, \\mu={mu}, k={k}$ // {name}')

    if i == 0:
        a.legend(
            loc='upper left',
            handles=[
                        matplotlib.patches.Patch(
                            color=f'C{server}',
                            alpha=0.5,
                            label=f'packets served by #{server + 1}'
                        )
                        for server in range(c)
                    ] + [
                        matplotlib.patches.Patch(
                            facecolor='white',
                            edgecolor=f'C{server}',
                            hatch='\\' * 5,
                            alpha=0.5,
                            label=f'packets discarded by #{server + 1}',
                        )
                        for server in range(c)
                    ],
        )

pass

## Total packets

In this section, we will see overall how many packets in total are processed by each server during the entire simulation runtime.

In [ ]:
axs: list[plt.Axes]
_, axs = plt.subplots(len(policies), 1, figsize=(12, 5 * len(policies) + 2), sharex='all', sharey='all')

for i, a, (name, policy) in zip(it.count(), axs, policies.items()):
    timestamps_packets: Iterable[tuple[float, NDArray[...]]] = timestamp_tot_packets(
        c,
        mmck_simulation(seed_arr, seeds_dep, lmbd, mu, k, policy)
    )

    samples: list[tuple[float, NDArray[...]]] = mit.take(100_000, timestamps_packets)
    step: int = 1_000
    _, pckts = zip(*samples)
    pckts = np.asarray(pckts, int)[::step, :]

    xs: NDArray[int] = np.arange(len(pckts)) + 1
    width: float = 0.5 / c

    for server in range(c):
        a.bar(
            xs + width * server,
            pckts[:, server, 0],
            width,
            color=f'C{server}',
            align='edge',
        )
        a.bar(
            xs + width * server,
            pckts[:, server, 1] * -1,
            width,
            color=f'C{server}',
            align='edge',
            hatch='\\' * 5,
        )

    a.axhline(
        0,
        color='red',
        linestyle='--'
    )

    a.grid(True, axis='y')
    a.set_axisbelow(True)
    a.set_xticks(xs[::10], (xs[::10] * step).astype(str))
    a.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
    a.set_xlabel('arrivals')
    a.set_ylabel('packets')
    a.set_title(f'$({seed_arr}, {seeds_dep}) \\vdash \\lambda={lmbd}, \\mu={mu}, k={k}$ // {name}')

    if i == 0:
        a.legend(
            loc='upper left',
            handles=[
                        matplotlib.patches.Patch(
                            color=f'C{server}',
                            label=f'tot packets served by #{server + 1}'
                        )
                        for server in range(c)
                    ] + [
                        matplotlib.patches.Patch(
                            edgecolor='black',
                            facecolor=f'C{server}',
                            hatch='\\' * 5,
                            label=f'tot packets discarded by #{server + 1}',
                        )
                        for server in range(c)
                    ],
        )

pass

In [ ]:
policy2pckts: dict[str, NDArray[...]] = {
    name: pckts
    for name, policy in policies.items()
    for simulation in [mmck_simulation(seed_arr, seeds_dep, lmbd, mu, k, policy)]
    for timestamps_packets in [timestamp_tot_packets(c, simulation)]
    for _, pckts in [mit.nth(timestamps_packets, 100_000)]
}

f: plt.Figure
axs: list[plt.Axes]
f, axs = plt.subplots(1, len(policies), figsize=(12, 5), sharey='all')

for i, a, (name, pckts) in zip(it.count(), axs, policy2pckts.items()):
    bars = a.bar(
        range(c),
        pckts[:, 0],
        color=[f'C{server}' for server in range(c)],
        label=[f'tot packets served by #{server + 1}' for server in range(c)],
    )
    a.bar_label(bars)

    bars = a.bar(
        range(c),
        pckts[:, 1],
        color=[f'C{server}' for server in range(c)],
        hatch='\\' * 5,
        label=[f'tot packets discarded by #{server + 1}' for server in range(c)],
    )
    a.bar_label(bars)

    a.yaxis.set_major_locator(matplotlib.ticker.MaxNLocator(integer=True))
    a.set_ylabel('packets')
    a.set_title(name)

    if i == 0:
        a.legend(loc='center')

f.suptitle(f'$({seed_arr}, {seeds_dep}) \\vdash \\lambda={lmbd}, \\mu={mu}, k={k}$')

pass

### Non-overlapping batches

To better understand the behaviour of the policies, we will measure the mean of the waiting time.
This will be done by simulating a long run, retrieving the metrics with non-overlapping batches to calculate a CI for the mean.

In [ ]:
m: int = 1_000
b: int = 200

batchess: dict[str, NDArray[float]] = {
    name: batches[:, :, 2]
    for name, policy in policies.items()
    for simulation in [mmck_simulation(seed_arr, seeds_dep, lmbd, mu, k, policy)]
    for ws in [waiting(c, simulation)]
    for batches in [non_overlapping_batches(ws, b, m)]
}

In [ ]:
muss: dict[str, NDArray[float]] = {
    name: np.average(batches, axis=1)
    for name, batches in batchess.items()
}
muss

In [ ]:
grand_means_v_delta: dict[str, NDArray[float]] = {
    name: np.asarray([grand_mean, v, delta])
    for name, mus in muss.items()
    for grand_mean in [st.fmean(mus)]
    for v in [np.sum((mus - grand_mean) ** 2) / (b - 1)]
    for delta in [sp.stats.t.ppf((1 + gamma()) / 2, df=b - 1) * math.sqrt(v / b)]
}
grand_means_v_delta

In [ ]:
rows: int = len(policies) + 1

axs: list[plt.Axes]
_, axs = plt.subplots(rows, 1, figsize=(12, 5 * rows + 1), sharex='all', )  # sharey='all')

for i, a, name in zip(it.count(), axs, policies.keys()):
    mus = muss[name]
    grand_mean, _, delta = grand_means_v_delta[name]
    populate(a, b, m, mus, grand_mean, (seed_arr, seeds_dep), lmbd, mu, k, delta,
             r'$\mathbf{E}\left[{\rm awaited}\right]$', name, i)

    axs[-1].axhspan(
        grand_mean - delta,
        grand_mean + delta,
        alpha=0.5,
        color=f'C{i}',
    )
    axs[-1].axhline(
        grand_mean,
        color=f'C{i}',
    )
    axs[-1].set_ylabel(r'$\mathbf{E}\left[{\rm awaited}\right]$')
    axs[-1].set_xlabel('samples')

pass

Note: the metrics above are **not** comparable

We need to take note that there in this simulation, the queue size is limited.
A packet that was supposed to be assigned to a full queue will be discarded.
A discarded packet will not be counted for waiting time.
A policy which always discard would measure no waiting time.

To make this comparison fair we repeat the simulation with no limitation on the queue size.

In [ ]:
c: int = 2
k: int = -1
lmbd: float = 95 * c
mu: float = 100

policies: dict[str, Callable] = {
    'round robin': round_robin,
    'least loaded': least_loaded,
    'least loaded round robin': least_loaded_round_robin,
}

In [ ]:
m: int = 1_000
b: int = 200

batchess: dict[str, NDArray[float]] = {
    name: batches[:, :, 2]
    for name, policy in policies.items()
    for simulation in [mmck_simulation(seed_arr, seeds_dep, lmbd, mu, k, policy)]
    for ws in [waiting(c, simulation)]
    for batches in [non_overlapping_batches(ws, b, m)]
}

In [ ]:
muss: dict[str, NDArray[float]] = {
    name: np.average(batches, axis=1)
    for name, batches in batchess.items()
}
muss

In [ ]:
grand_means_v_delta: dict[str, NDArray[float]] = {
    name: np.asarray([grand_mean, v, delta])
    for name, mus in muss.items()
    for grand_mean in [st.fmean(mus)]
    for v in [np.sum((mus - grand_mean) ** 2) / (b - 1)]
    for delta in [sp.stats.t.ppf((1 + gamma()) / 2, df=b - 1) * math.sqrt(v / b)]
}
grand_means_v_delta

In [ ]:
rows: int = len(policies) + 1

axs: list[plt.Axes]
_, axs = plt.subplots(rows, 1, figsize=(12, 5 * rows + 1), sharex='all', )  # sharey='all')

for i, a, name in zip(it.count(), axs, policies.keys()):
    mus = muss[name]
    grand_mean, _, delta = grand_means_v_delta[name]
    populate(a, b, m, mus, grand_mean, (seed_arr, seeds_dep), lmbd, mu, k, delta,
             r'$\mathbf{E}\left[{\rm awaited}\right]$', name, i)

    axs[-1].axhspan(
        grand_mean - delta,
        grand_mean + delta,
        alpha=0.5,
        color=f'C{i}',
    )
    axs[-1].axhline(
        grand_mean,
        color=f'C{i}',
    )
    axs[-1].set_ylabel(r'$\mathbf{E}\left[{\rm awaited}\right]$')
    axs[-1].set_xlabel('samples')

pass

We can see from the plot above that the expected waiting time for round robin policy is higher than other policies.
We should be able to infer that round robin policy performs worse compared to other, but we would like to calculate additional measurement to ensure.

$$
\left[ \overline Z_{1} - \overline Z_{2} \pm t_{2 (n - 1), \frac {1 + \gamma} 2} \sqrt{\frac{V_1 + V_2}{b}} \right]_\gamma
$$

In [ ]:
for p1, p2 in it.combinations(policies.keys(), 2):
    grand_mean1, v1, _ = grand_means_v_delta[p1]
    grand_mean2, v2, _ = grand_means_v_delta[p2]

    estimation = grand_mean1 - grand_mean2
    delta = sp.stats.t.ppf((1 + gamma()) / 2, df=b * m * 2 - 2) * math.sqrt((v1 + v2) / b)

    print(p1 + " vs " + p2)
    print(estimation)
    print(delta)

We can see that for both round robin against least loaded and round robin against least loaded round robin, the CIs are greater than zero, this means that round robin performs worse against the other policies.

On the other hand, for least loaded against least loaded round robin, zero lies within the range of both CIs.
This means that we cannot conclusively decide if one is better than the other.